## Tools

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.tools import BaseTool, FunctionTool

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [ ]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

In [ ]:
print(response.sources)

## Using Query Engine

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.agent import OpenAIAgent
from llama_index.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

In [ ]:
agent.chat_repl()

## Context Augmented Agent

In [ ]:
import json
from typing import Sequence

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.tools import BaseTool, FunctionTool

from llama_index.schema import Document
from llama_index.agent import ContextRetrieverOpenAIAgent

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/march"
    )
    march_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/june"
    )
    june_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/sept"
    )
    sept_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
# build indexes across the three data sources

if not index_loaded:
    # load data
    print("Loading data...")
    march_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_march_2022.pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_june_2022.pdf"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_sept_2022.pdf"]
    ).load_data()
    print("Done!\n\n")

    # build index
    print("Building index...")
    march_index = VectorStoreIndex.from_documents(march_docs)
    june_index = VectorStoreIndex.from_documents(june_docs)
    sept_index = VectorStoreIndex.from_documents(sept_docs)
    print("Done!\n\n")

    # persist index
    print("Persisting index...")
    march_index.storage_context.persist(persist_dir="./storage/march")
    june_index.storage_context.persist(persist_dir="./storage/june")
    sept_index.storage_context.persist(persist_dir="./storage/sept")
    print("Done!\n\n")

In [ ]:
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=march_engine,
        metadata=ToolMetadata(
            name="uber_march_10q",
            description=(
                "Provides information about Uber 10Q filings for March 2022. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=june_engine,
        metadata=ToolMetadata(
            name="uber_june_10q",
            description=(
                "Provides information about Uber financials for June 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=sept_engine,
        metadata=ToolMetadata(
            name="uber_sept_10q",
            description=(
                "Provides information about Uber financials for Sept 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
# toy index - stores a list of abbreviations
texts = [
    "Abbreviation: X = Revenue",
    "Abbreviation: YZ = Risk Factors",
    "Abbreviation: Z = Costs",
]
docs = [Document(text=t) for t in texts]
context_index = VectorStoreIndex.from_documents(docs)

In [ ]:
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    query_engine_tools,
    context_index.as_retriever(similarity_top_k=1),
    verbose=True,
)

In [ ]:
response = context_agent.chat("What is the YZ of March 2022?")

In [ ]:
print(str(response))

In [ ]:
context_agent.chat("What is the X and Z in September 2022?")

In [ ]:
def magic_formula(revenue: int, cost: int) -> int:
    """Runs MAGIC_FORMULA on revenue and cost."""
    return revenue - cost


magic_tool = FunctionTool.from_defaults(fn=magic_formula, name="magic_formula")

In [ ]:
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    [magic_tool], sept_index.as_retriever(similarity_top_k=3), verbose=True
)

In [ ]:
response = context_agent.chat(
    "Please run MAGIC_FORMULA on Uber's revenue and cost"
)

In [ ]:
print(response)

## Context from webpage

In [39]:
from llama_index import VectorStoreIndex, StorageContext, download_loader, load_index_from_storage
from llama_index.schema import Document
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.agent import ContextRetrieverOpenAIAgent
from llama_index.prompts import PromptTemplate

In [21]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/zharta"
    )
    index_gitbook = load_index_from_storage(storage_context)
    query_engine_gitbook = index_gitbook.as_query_engine()

    print("All data loaded from disk!")
    index_loaded = True
except:
    index_loaded = False
    print("Data not loaded from disk!")

All data loaded from disk!


In [15]:
if not index_loaded:
    print("Reading gitbook...")
    TrafilaturaWebReader = download_loader("TrafilaturaWebReader")

    loader = TrafilaturaWebReader()
    documents_gitbook = loader.load_data(
        urls=[
            'https://zharta.gitbook.io/zharta-welcome-kit/overview/introducing-zharta',
            'https://zharta.gitbook.io/zharta-welcome-kit/overview/manifesto',
            'https://zharta.gitbook.io/zharta-welcome-kit/overview/team',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/zharta-lending-offer',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/supported-collections',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/faq',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-borrower',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-borrower/requesting-a-loan',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-borrower/grace-period',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-borrower/notification-system',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-borrower/delegation',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-borrower/tailored-deals',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-lender',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-lender/making-a-deposit',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-lender/lending-pools',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/as-a-lender/buy-now',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management/data',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management/trait-boosts',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management/stability-controls',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management/risk-monitoring',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management/recovery-liquidation',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/risk-management/protocol-simulator',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/protocol-lending',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/protocol-lending/contract-addresses',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/protocol-lending/protocol-fees',
            'https://zharta.gitbook.io/zharta-welcome-kit/nft-lending/protocol-lending/security-audits',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/zharta-renting-offer',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/supported-collections-renting',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/faq',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/v2-migration',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-an-owner',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-an-owner/listing-an-asset',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-an-owner/updating-rental-conditions',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-an-owner/self-delegation',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-an-owner/unlisting-and-withdrawing',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-an-owner/claiming-your-earnings',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-a-renter',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-a-renter/renting-an-asset',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-a-renter/rental-delegation',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/as-a-renter/canceling-a-rental',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/protocol-renting',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/protocol-renting/protocol-fees',
            'https://zharta.gitbook.io/zharta-welcome-kit/otherside-renting/protocol-renting/security-audits',
            'https://zharta.gitbook.io/zharta-welcome-kit/resources/privacy-policy',
            'https://zharta.gitbook.io/zharta-welcome-kit/resources/terms-and-conditions',
            'https://zharta.gitbook.io/zharta-welcome-kit/resources/support',
            'https://zharta.gitbook.io/zharta-welcome-kit/resources/terminology',
        ]
    )
    index_gitbook = VectorStoreIndex.from_documents(documents_gitbook)
    index_gitbook.storage_context.persist(persist_dir="./storage/zharta")
    query_engine_gitbook = index_gitbook.as_query_engine()
    print("Done!")
else:
    print("Index loaded from disk")

Index loaded from disk


In [ ]:
if not index_loaded:
    # load data
    print("Loading data...")
    march_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_march_2022.pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_june_2022.pdf"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["./data/10q/uber_10q_sept_2022.pdf"]
    ).load_data()
    print("Done!\n\n")

    # build index
    print("Building index...")
    march_index = VectorStoreIndex.from_documents(march_docs)
    june_index = VectorStoreIndex.from_documents(june_docs)
    sept_index = VectorStoreIndex.from_documents(sept_docs)
    print("Done!\n\n")

    # persist index
    print("Persisting index...")
    march_index.storage_context.persist(persist_dir="./storage/march")
    june_index.storage_context.persist(persist_dir="./storage/june")
    sept_index.storage_context.persist(persist_dir="./storage/sept")
    print("Done!\n\n")

In [16]:
documents_contacts = [
    Document(text="Zharta's Email Address is test@test.com"),
    Document(text="Zharta's X account is @zhartafinance"),
    Document(text="Zharta's Discord server is @randomcenas"),
]
index_contacts = VectorStoreIndex.from_documents(documents_contacts)
query_engine_contacts = index_contacts.as_query_engine()

In [33]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_contacts,
        metadata=ToolMetadata(
            name="contacts",
            description=(
                "Provides information about Zharta's contacts. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [34]:
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    query_engine_tools,
    index_gitbook.as_retriever(similarity_top_k=3),
    verbose=True,
)

In [41]:
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

context_str = (
    'Always answer as if you were a customer service answer working at Zharta. '
    'Prefer to use "we" instead of "them" when talking about Zharta, its team or its customer service. '
    'Always keep in mind that Zharta provides two services: lending and borrowing using NFTs as collateral, and renting for the LOTM game. '
    'Never abbreviate Legens of the Mara as LOTM. '
)

In [57]:
user_query = "why do you say that zharta has a token?"

prompt = qa_template.format(context_str=context_str, query_str=user_query)
response = context_agent.chat(prompt)

Context information is below.
---------------------
Zharta Documentation
Search
⌃
K
OVERVIEW
Introducing Zharta
Manifesto
Team
NFT Lending
Zharta Lending Offer
Supported Collections
FAQ
As a Borrower
As a Lender
Risk Management
Protocol - Lending
Otherside Renting
Zharta Renting Offer
Supported Collections – Renting
FAQ
V2 Migration
As an Owner
As a Renter
Protocol - Renting
Resources
Privacy Policy
Terms & Conditions
Support
Terminology
Powered By
GitBook
Comment on page
Introducing Zharta
DeFi Web3 startup that provides instant loans & renting services using NFTs as collateral, enabling NFT holders to get liquidity & yield without having to sell their NFTs
Why Zharta?
1.
Instant Liquidity
2.
Liquidation Protection
3.
Flexible, Pro-Rata Pricing
4.
Asset Bundling
5.
Exclusive Lender Access to Defaulted NFTs
6.
Otherside Renting
Next - OVERVIEW
Manifesto
Last modified
12d ago
Comment on page
FAQ
What the ... ?
Zharta is a real-time NFT loan platform for borrowers and lenders.
Our smart 

In [58]:
print(str(response))

Zharta does not currently have its own token. It is a DeFi Web3 startup that provides instant loans and renting services using NFTs as collateral. While Zharta operates on the Ethereum Network and interacts with cryptocurrencies like Wrapped Ether (WETH), it does not have its own native token.
